In [1]:
# Run using biosimspace-dev conda env

In [2]:
import BioSimSpace as BSS


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/nichrun/miniconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/nichrun/miniconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/nichrun/miniconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/nichrun/miniconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/nichrun/

# Get paramaterised molecules

In [3]:
import glob
path_to_cluster = "../clusters/cls-22-06-27-2/"
num_array = [x.split("/")[-1].split(".")[0] for x in glob.glob(path_to_cluster+"*.rst7")]

molecules_amber = {}
for num in num_array:
    molecules_amber[int(num)] = (BSS.IO.readMolecules([f"{path_to_cluster}{num}.rst7", f"{path_to_cluster}{num}.prm7"]).getMolecule(0))

In [5]:
molecules_amber

{15696: <BioSimSpace.Molecule: nAtoms=65, nResidues=1>,
 15694: <BioSimSpace.Molecule: nAtoms=69, nResidues=1>,
 15697: <BioSimSpace.Molecule: nAtoms=65, nResidues=1>,
 14330: <BioSimSpace.Molecule: nAtoms=68, nResidues=1>,
 16393: <BioSimSpace.Molecule: nAtoms=64, nResidues=1>,
 14336: <BioSimSpace.Molecule: nAtoms=67, nResidues=1>,
 10920: <BioSimSpace.Molecule: nAtoms=67, nResidues=1>,
 17333: <BioSimSpace.Molecule: nAtoms=55, nResidues=1>,
 14347: <BioSimSpace.Molecule: nAtoms=59, nResidues=1>,
 14348: <BioSimSpace.Molecule: nAtoms=60, nResidues=1>,
 17334: <BioSimSpace.Molecule: nAtoms=54, nResidues=1>,
 14349: <BioSimSpace.Molecule: nAtoms=56, nResidues=1>}

# Create lambda files

In [7]:
pert_pairs = [(14349, 15697), #Already complete
    (10920,14330), # 0.59
    (10920,14347), # 1.0
    (10920,15696), # 0.72
    (16393,15697), # 0.76
    (14349,14348), # 0.66
    (16393,17333), # 0.2
    (15696,14347), # 0.15
    (17334,14336)] # 0.18
pert_pairs

[(10920, 14330),
 (10920, 14347),
 (10920, 15696),
 (16393, 15697),
 (14349, 14348),
 (16393, 17333),
 (15696, 14347),
 (17334, 14336)]

In [10]:
import os

#pert_pair = (14349, 15697)
for pert_pair in pert_pairs[1:]:
    path_to_lambda = f"../fep_lambda_files/cls-22-06-27-2/{pert_pair[0]}-{pert_pair[1]}/"

    if not os.path.exists(path_to_lambda):
        os.makedirs(path_to_lambda)
        
    mol1 = molecules_amber[pert_pair[0]]
    mol2 = molecules_amber[pert_pair[1]]

    mapping = BSS.Align.matchAtoms(mol1, mol2)
    mol1 = BSS.Align.rmsdAlign(mol1, mol2, mapping)

    merged = BSS.Align.merge(mol1, mol2, mapping, 
                             allow_ring_breaking=True,
                             #allow_ring_size_change=True
                            )

    solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom])
    protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=11)
    
    
    for run_num in range(0,6):
        fep_free = BSS.FreeEnergy.Relative(solvated, protocol, work_dir=f"{path_to_lambda}/run{run_num}/free")
        fep_vac  = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, work_dir=f"{path_to_lambda}/run{run_num}/vacuum")

In [31]:
"""
mol1 = molecules_amber[pert_pair[0]]
mol2 = molecules_amber[pert_pair[1]]

mapping = BSS.Align.matchAtoms(mol1, mol2)
mol1 = BSS.Align.rmsdAlign(mol1, mol2, mapping)

merged = BSS.Align.merge(mol1, mol2, mapping, 
                         allow_ring_breaking=True,
                         #allow_ring_size_change=True
                        )

solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom])
protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=11)

#Can loop over these two lines to duplicate
"""

In [34]:
"""
for run_num in range(0,6):
    fep_free = BSS.FreeEnergy.Relative(solvated, protocol, work_dir=f"{path_to_lambda}/run{run_num}/free")
    fep_vac  = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, work_dir=f"{path_to_lambda}/run{run_num}/vacuum")
"""